In [1]:
from opensearchpy import OpenSearch
from opensearchpy.helpers import bulk
from opensearch_dsl import Search
import requests
import json
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import requests
def get_access_token():
    auth_url = 'https://hp-gsb.docebosaas.com/manage/v1/user/login'
    user_credentials = {
        'username': 'apiknowledgecentral@hp.com',
        'password': '5w62992'
    }
    try:
        response = requests.post(auth_url, data=user_credentials)
        response.raise_for_status()
        auth_data = response.json()
        print(auth_data)
        access_token = response.json()["data"]["access_token"]
        if access_token:
            return access_token
        else:
            print("Error: Access token not found in response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error in authentication request: {e}")
        return None

access_token = get_access_token()
print( access_token)

{'data': {'access_token': '22004369073fa9cf33781bf52d65ee4cb3a5c764', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}
22004369073fa9cf33781bf52d65ee4cb3a5c764


In [3]:
def doccebo_get_access_token():
    config=docceboAPIConfig()
    auth_url = config.auth_url
    user_credentials = {
        'username': config.auth_username,
        'password': config.auth_password
    }
    try:
        response = requests.post(auth_url, data=user_credentials)
        response.raise_for_status()
        auth_data = response.json()
        access_token = response.json()["data"]["access_token"]
        if access_token:
            return access_token
        else:
            print("Error: Access token not found in response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error in authentication request: {e}")
        return None

In [5]:
a=doccebo_get_access_token()
a

'60987edc48b9fa6c609cfe6ed8e760e0482c1ee8'

In [4]:
class OpenSearchConnectConfig:
    def __init__(self):
        self.host = 'vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com'       
        self.port = 443
        self.auth = ('kcs-admin', '!QAZ2wsx')    
        self.use_ssl = True
        self.verify_certs = False
        
class OpenSearchConnectConfigDev:
    def __init__(self):
        self.host = 'vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com' 
       # https://vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com/
        self.port = 443
        self.auth = ('ks-stg-admin', 'K3!0dg_7Hc')    
        self.use_ssl = True
        self.verify_certs = False
        
class OpenSearchConnectConfigStg:
    def __init__(self):
        self.host = 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com' 
       # https://vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com/
        self.port = 443
        self.auth = ('ks-stg-admin', 'K3!0dg_7Hc')    
        self.use_ssl = True
        self.verify_certs = False

class docceboAPIConfig:
    def __init__(self):
        self.auth_url = 'https://hp-gsb.docebosaas.com/manage/v1/user/login'  
        self.auth_username = 'apiknowledgecentral@hp.com'            
        self.auth_password = '5w62992'  
        self.doccebo_course_delta_url = 'https://hp-gsb.docebosaas.com/learn/v1/courses'
        self.doccebo_catagories_data_url ='https://hp-gsb.docebosaas.com/learn/v1/categories'
        self.doccebo_catalog_data = 'https://hp-gsb.docebosaas.com/learn/v1/catalog'
        self.doccebo_course_data = 'https://hp-gsb.docebosaas.com/learn/v1/courses'

config=docceboAPIConfig()

In [6]:
# logging
import os
import datetime
import logging

def get_current_date():
    return datetime.datetime.now().strftime('%Y-%m-%d')


class LogConfig:
    """
    Configuration class for logging settings.
    """

    def __init__(self, log_dir="PROJECT_logs", log_level=logging.INFO, format_string="[%(asctime)s]^;%(levelname)s^;%(lineno)d^;%(filename)s^;%(funcName)s()^;%(message)s"):
        self.log_dir = log_dir
        self.log_level = log_level
        self.format_string = format_string

def get_log_file_name(prefix, date):
    """
    Generates a log file name with a given prefix and date.
    """
    return f"log_{prefix}_{date}.log"

def setup_logging(config=LogConfig()):
    """
    Sets up logging configuration based on provided LogConfig object.

    Args:
        config (LogConfig, optional): Configuration object with desired settings. Defaults to LogConfig().
    """
    os.makedirs(config.log_dir, exist_ok=True)

    # Determine current date
    current_date = get_current_date()

    # Set up all log file path
    all_log_file_path = os.path.join(config.log_dir, get_log_file_name(prefix="all", date=current_date))
    error_log_file_path = os.path.join(config.log_dir, get_log_file_name(prefix="error", date=current_date))

    # Set up all log handler
    all_handler = logging.FileHandler(all_log_file_path, mode="a")  # Append mode
    all_handler.setLevel(logging.DEBUG)
    all_handler.setFormatter(logging.Formatter(config.format_string))

    # Set up error log handler
    error_handler = logging.FileHandler(error_log_file_path, mode="a")  # Append mode
    error_handler.setLevel(logging.ERROR)
    error_handler.setFormatter(logging.Formatter(config.format_string))

    # Root logger configuration
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.DEBUG)
    root_logger.addHandler(all_handler)
    root_logger.addHandler(error_handler)

def get_log_dataframe(file_path):
    data = []
    with open(file_path) as log_file:
        for line in log_file.readlines():
            data.append(line.split("^;"))

    log_df = pd.DataFrame(data)
    columns = ["Time stamp", "Log Level", "Line number", "File name", "Function name", "Message"]
    log_df.columns = columns
    
    log_df["log_message"] = log_df['Time stamp'].astype(str) + ": $" + log_df["Message"]

    return log_df[["log_message"]]


In [8]:
#INC8510707    'https://hp-gsb.docebosaas.com/learn/v1/courses'

In [ ]:
DFE, Workflow, PrintOS	34
General	159
HP Indigo Digital Press	38
Production Management	213
Trainer	39
HP Indigo & PWP Sales (Non-GSB) - AMS	313
Indigo Testing Center	190
Indigo Service	2
PageWide Industrial	9
PageWide Industrial Post Print (Scitex)	163

In [99]:
# Step 1: Perform the merge and aggregate item_name directly in the grouped DataFrame
test = pd.merge(
    df,
    cat_df,
    how="left",
    left_on="code",
    right_on="code",
)

# Step 2: Aggregate item_name as a list while retaining all columns
test['item_name'] = test.groupby('documentID')['item_name'].transform(lambda x: list(x))

# Step 3: Drop duplicates to get one row per documentID
test2 = test.drop_duplicates(subset='documentID').reset_index(drop=True)


In [206]:
for i,j in df['products'].value_counts().items():
    print(i,j)

[] 1446
['HP Indigo 1x000 Digital Press'] 121
['HP Indigo WS6xxx Digital Press'] 94
['HP Indigo 7xxx Digital Press'] 92
['HP PWP T2'] 82
['HP Indigo 20000 Digital Press'] 81
['HP PWP T4'] 75
['HP Indigo 100K Digital Press'] 69
['HP PWP T3'] 66
['HP Indigo 10000 Digital Press', 'HP Indigo 10000HD Digital Press', 'HP Indigo 10r Digital Press', 'HP Indigo 12000 Digital Press', 'HP Indigo 12000HD Digital Press', 'HP Indigo WS6600 Digital Press', 'HP Indigo WS6600p Digital Press', 'HP Indigo WS6800p Digital Press', 'HP Indigo 6900 Digital Press', 'HP Indigo WS6800 Digital Press', 'HP Indigo 7600 Digital Press', 'HP Indigo 7800 Digital Press', 'HP Indigo 7000 Digital Press', 'HP Indigo 7500 Digital Press', 'HP Indigo 7r Digital Press', 'HP Indigo 20000 Digital Press', 'HP Indigo 8000 Digital Press', 'HP Indigo 30000 Digital Press', 'HP Indigo 5600 Digital Press', 'HP Indigo 5900 Digital Press', 'HP Indigo 5000 Digital Press', 'HP Indigo 3500 Digital Press', 'HP Indigo 3550 Digital Press', 'H

In [ ]:
    def fetch_doccebo_course_data(self):
        # Define API endpoint URL
        base_url = "https://hp-gsb.docebosaas.com/learn/v1/courses"
        # Define parameters (filters)
        params = {
            "updated_from": DoceboCourseDeltaLoader.fromDate,
            "page_size": 10,
            'adv': '1'
        }
        
        try:
            docebo_bearer = doccebo_get_access_token()
            page = 1
            data = []
        
            while True:
                # Add page parameter to the params dictionary
                params['page'] = page
        
                # Make the GET request
                response = requests.get(base_url, params=params, headers={"Authorization": "Bearer " + docebo_bearer})
    
                if response.status_code == 200:
                    response_json = response.json()
                    data.extend(response_json['data']['items'])
                    total_pages = response_json['data']['total_page_count']
                    
                    if page >= total_pages:
                        break  # Break the loop if we've fetched all pages
                    else:
                        page += 1  # Move to the next page
                else:
                    logging.error("Failed to fetch data from page %d. Status code: %d", page, response.status_code)
                    break
    
            return data
        
        except Exception as e:
            logging.error("An error occurred: %s", str(e))
            return None

In [6]:
import pandas as pd
import datetime
import ast
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
from bs4 import BeautifulSoup

class DoceboCourseDeltaLoader:
    fromDate = '2024-08-25 00:00:00'  # Class attribute for the fromDate
    failed_records_df = pd.DataFrame(columns=['Record', 'Error', 'Source'])
    def fetch_doccebo_course_data(self):
        # Define API endpoint URL
        base_url = "https://hp-gsb.docebosaas.com/learn/v1/courses"
        # Define parameters (filters)
        params = {
            # "updated_from": DoceboCourseDeltaLoader.fromDate,
            "page_size": 10,
            'adv': '1'
        }
    
        docebo_bearer = doccebo_get_access_token()
    
        # Make initial request to determine total pages
        initial_response = requests.get(base_url, params=params, headers={"Authorization": "Bearer " + docebo_bearer})
        initial_response.raise_for_status()
        initial_data = initial_response.json()
        total_pages = initial_data['data']['total_page_count']
        data = initial_data['data']['items']  # Collect the initial page
    
        def fetch_page_data(page):
            """Fetch a single page of Docebo course data."""
            page_params = params.copy()
            page_params['page'] = page
            response = requests.get(base_url, params=page_params, headers={"Authorization": "Bearer " + docebo_bearer})
            response.raise_for_status()
            return response.json()['data']['items']
    
        # Use ThreadPoolExecutor to fetch multiple pages in parallel
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(fetch_page_data, page) for page in range(2, total_pages + 1)]
            for future in as_completed(futures):
                data.extend(future.result())
        return data    
        
    def preprocessing(self, data):
        try:
            # Convert input data to DataFrame
            df = pd.DataFrame(data)
            
            # Rename columns for consistency
            updated_columns = {
                'id_course': 'documentID',
                'name': 'title',
                'uidCourse': 'id',
                'date_last_updated': 'contentUpdateDate',
                'course_type': 'contentType'
            }
            df.rename(columns=updated_columns, inplace=True)
            
            # Drop unnecessary columns if they exist in the DataFrame
            columns_to_drop = [
                'start_date', 'end_date', 'duration', 
                'available_seats', 'available_seats_course'
            ]
            columns_to_drop = [col for col in columns_to_drop if col in df.columns]
            df.drop(columns=columns_to_drop, inplace=True)
            
            # Fill missing values in specific columns with -1
            columns_to_fill = ['current_rating', 'max_attempts']
            df[columns_to_fill] = df[columns_to_fill].fillna(-1)
            
            df['description'] = df['description'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text(separator="\n").strip().replace("\n", " ").replace('•', '\n•'))
            
            # Combine title and description into a single column
            df['ti_desc_prod'] = df['title'].fillna('') + ' ' + df['description'].fillna('')
            df["ti_desc_prod"] = df["ti_desc_prod"].replace(r"^\s*$", "N/A", regex=True)

            # Create Doc_status field
            df["Doc_Status"] = df["ti_desc_prod"].apply(
                lambda x: "rejected" if x == "N/A" else "published"
            )
            
            # Mark the domain as 'Indigo' if the code contains the search string
            indigo_search_string = 'gsb.i'
            df.loc[df['code'].str.contains(indigo_search_string), 'Domain'] = 'Indigo'

            pwp_search_string = 'gsb.pwp'
            df.loc[df['code'].str.contains(pwp_search_string), 'Domain'] = 'PWP'
            
            # Assign persona based on the slug_name pattern
            # df['persona'] = df['slug_name'].apply(lambda x: "Engineer" if '-ce-' in x or '-fse-' in x or 'service-engineer' in x else "Operator")
            ##df['persona'] = df['slug_name'].apply(lambda x: "Engineer" if re.search(r'(\bce\b|\b-fse\b|\bce\b)', x.lower()) or 'service-engineer' in x.lower() else "Operator")
            
            # Filter rows where Domain is 'Indigo' or  'pwp'
            df = df[df['Domain'].isin(['Indigo', 'PWP'])]
            language_code_mapping = {
                'English': 'en',
                'EN': 'en',
                'En': 'en',
                'Chinese': 'zh',
                'French': 'fr',
                'German': 'de',
                'Japanese': 'ja',
                'Korean': 'ko',
                'Portuguese': 'pt',
                'Russian': 'ru',
                'Spanish': 'es',
                'Italian': 'it',
                'Simplified_Chinese': 'zh-CN',
                'Portuguese-Br': 'pt-BR',
                'Hebrew': 'he',
                'Spanish_Latam': 'es-419',
                'Hungarian': 'hu',
                'Dutch': 'nl',
                'Other Languages': 'xx'  # Handling other unspecified languages
            }
            
            # Strip whitespace and normalize case
            df['language'] = df['language'].str.strip().str.title()
            
            # Replace the language names with their corresponding ISO codes
            df['language'] = df['language'].replace(language_code_mapping)
            # Read the products mapping from an Excel file and merge with the main DataFrame
            #products_mapping = pd.read_csv('doccebo_products.csv')
            products_mapping=pd.read_csv('press_name_products.csv')
            
            products_mapping['products'] = products_mapping['products'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

            df = pd.merge(df, products_mapping, how='left', left_on='documentID',right_on='documentID')

            indigo_service_category=[  5.,  13.,  15.,  54.,  19.,  63.,  58.,  53.,  62., 160.,  70.,
                    72.,  69., 146., 159.,  39., 151., 120.,  38., 152., 309.,  34.,
                    50.,  49., 156., 158., 155., 164., 213., 186., 285., 118., 162.,
                   296., 295., 297., 301., 140., 141., 299., 166., 148., 183., 191.,
                   189., 205., 188., 239., 468., 242., 145., 244., 282.,  41., 431.,
                   394., 300., 331., 341., 358., 398., 393., 392., 411., 432., 413.,
                   294., 540., 509., 508., 592., 437., 514., 539., 525., 577., 511.,
                   596., 597., 601.]
            
            indigo_df=df[df['Domain'] == 'Indigo'].loc[
                lambda x: x['category'].apply(lambda y: y['id'] in indigo_service_category)
            ]
            pwp_df = df[df['Domain']=='PWP']
            df = pd.concat([indigo_df, pwp_df], ignore_index=True)
            
            # Fill any remaining missing values with empty strings
            render_url="https://hp-gsb.docebosaas.com/learn/course"
            df['renderLink']=df['documentID'].apply(lambda x: f"{render_url}/{x}")
            patterns = [r'^(HP Indigo.*?(Press(?:es)?|Series))', r'^(HP PWP.*?(Series|T\d+))']

            df.fillna('', inplace=True)
            df['products']=df['products'].apply(lambda x:[] if x==''else x)
            df['products'] = df.apply(
                lambda row: row['products'] if row['products'] not in [None, []] else [
                    match.group(0) for pattern in patterns 
                    if (match := re.search(pattern, row['title'], re.IGNORECASE)) 
                    and len(match.group(0).split()) < 7
                ],
                axis=1
            )
            df['products'] = df['products'].apply(
                lambda lst: [
                    f"{' '.join(parts[:2])} {model} {parts[-1]}"
                    for device in lst
                    if isinstance(device, str)
                    for parts in [device.split(' ')]
                    if len(parts) >= 3 and '/' in parts[2]
                    for model in parts[2].split('/')
                ] + [
                    device
                    for device in lst
                    if isinstance(device, str) and '/' not in device
                ]
                if isinstance(lst, list) else lst
            )
            df['products'] = df['products'].apply(lambda devices_list: [device.strip() for item in devices_list for device in item.split(',')] if all(isinstance(item, str) for item in devices_list) else devices_list)

            return df
        except Exception as e:
            logging.error("An error occurred during preprocessing: %s", str(e))
            return []
            
    def get_persona(self,df):
        
        course_ids = df['documentID'].tolist()
        attendee = {}
        access_token=get_access_token()
        # Function to make the API request and process the response for each course
        def fetch_course_data(i):
            api_endpoint = f'https://hp-gsb.docebosaas.com/learn/v1/courses/{i}'
            headers = {"Authorization": "Bearer " + access_token}
    
            response = requests.get(api_endpoint, headers=headers)
            if response.status_code == 200:
                response_json = response.json()
                data = response_json['data'].get("additional_fields", [])
                is_published = response_json['data'].get('is_published', 'N/A')
                press_name_dict = next((item for item in data if item['title'] == 'Attendee Type'), None)
    
                if press_name_dict:
                    return i, {
                        'title': press_name_dict.get('title', 'N/A'),
                        'value': press_name_dict.get('value', 'N/A'),
                        'is_published': is_published
                    }
                else:
                    print(f"Course ID: {i} - 'Attendee Type' not found in additional fields")
                    return i, {'title': 'Attendee Type', 'value': 'N/A'}
            else:
                print(f"Course ID: {i} - Request failed with status code {response.status_code}")
                return i, {'title': 'Attendee Type', 'value': 'N/A'}
    
        # Parallel processing using ThreadPoolExecutor
        with concurrent.futures.ThreadPoolExecutor(max_workers=25) as executor:
            futures = {executor.submit(fetch_course_data, course_id): course_id for course_id in course_ids}
    
            for future in concurrent.futures.as_completed(futures):
                course_id, result = future.result()
                attendee[course_id] = result
    
        # Creating a DataFrame with results
        attendee_df = pd.DataFrame(
            [(k, v['value'], v.get('is_published', 'N/A')) for k, v in attendee.items()],
            columns=['documentID', 'attendee_type', 'is_published']
        )
        updated_df=pd.merge(df,attendee_df,how="left",left_on="documentID",right_on="documentID")
        updated_df['persona']=updated_df['attendee_type'].apply(lambda x:'Engineer' if x=='HP Employee / Staff' else 'Operator' if x=='Customer' else 'No_roles')
        updated_df['persona'] = updated_df.apply(lambda row: 
            "Engineer" if row['persona'] == 'No_roles' and (re.search(r'(\bce\b|\b-fse\b|\bce\b)', row['slug_name'].lower()) or 'service-engineer' in row['slug_name'].lower()) 
            else "Operator" if row['persona'] == 'No_roles' 
            else row['persona'], axis=1) 
        updated_df=updated_df[updated_df['is_published']==True]
        return updated_df
        
    def load_doccebo_course_data_test(self, df, index_name):
        
        # Connect to OpenSearch
        config1 = OpenSearchConnectConfigStg()
        timeout_seconds = 160
        client = OpenSearch(
            hosts=[{'host': config1.host, 'port': config1.port}],
            http_compress=True,
            http_auth=config1.auth,
            use_ssl=config1.use_ssl,
            verify_certs=config1.verify_certs,
            timeout=timeout_seconds
        )
    
        dict_list = df.to_dict(orient="records")
        document_ids = [record["id"] for record in dict_list]
        
        # Batch fetch existing documents
        existing_docs = fetch_existing_documents(client, index_name, document_ids)
    
        # Initialize counters and lists
        successful_records = 0
        failed_records = []
        new_records = 0
        updated_records = 0
    
        # Prepare actions for bulk indexing with upsert
        actions = []
        for record in dict_list:
            if existing_docs.get(record["id"], False):
                updated_records += 1
            else:
                new_records += 1
    
            action = {
                "_index": index_name,
                "_id": record["id"],
                "pipeline": "doccebo.v2.emb_pipeline",
                "_op_type": "update",
                "doc": record,
                "doc_as_upsert": True,
            }
            actions.append(action)
    
        # Perform bulk indexing with upsert
        try:
            success, _ = bulk(client, actions=actions)
            successful_records = success
        except Exception as e:
            failed_records.append((record, str(e), "'DOCCEBO_COURSE_DELTA_API'"))
            logging.error(
                    f"Failed to index document {record}: {e}  : ''DOCCEBO_COURSE_DELTA_API'_INGRESS'"
                )
    
        # Store failed records and update fromDate if necessary
        if failed_records:
            logging.info(
                f"Indexing completed with {len(failed_records)} failed records."
            )
            self.store_failed_records(failed_records)
        else:
            # Update fromDate attribute if all records are successful
            current_datetime = datetime.datetime.now()
            DoceboCourseDeltaLoader.fromDate = current_datetime.strftime("%Y-%m-%d")
            logging.info("All records indexed successfully. Updating fromDate.")
    
        # Log indexing statistics to 'kas_log_index' index
        try:
            data_to_index = {
                "successful_records": successful_records,
                "failed_records": len(failed_records),
                "new_records": new_records,
                "updated_records": updated_records,
                "timestamp": datetime.datetime.now(),
                "Source": 'DOCCEBO_API_COURSE_DATA_INGESTION',
                "index_loaded": index_name,
            }
            client.index(index="doccebo_course_delta_log_index", body=data_to_index)
            logging.info("Successfully indexed data to doccebo_course_delta_log_index.")
        except Exception as e:
            logging.error(f"Failed to index data to doccebo_course_delta_log_index: {e}")
        logging.info(
            "successful_records: %s ::: failed_records: %s",
            successful_records,
            failed_records,
        )
    
        return successful_records, len(failed_records)

    def store_failed_records(self, failed_records):
        try:
            # Check if the DataFrame is empty
            if len(DoceboCourseDeltaLoader.failed_records_df) == 0:
                # Initialize failed_records_df if it's empty
                DoceboCourseDeltaLoader.failed_records_df = pd.DataFrame(columns=['Record', 'Error', 'Source'])
                
            # Convert failed_records to DataFrame
            failed_records_df_new = pd.DataFrame(failed_records, columns=['Record', 'Error', 'Source'])
            
            # Concatenate new records with existing DataFrame
            DoceboCourseDeltaLoader.failed_records_df = pd.concat([DoceboCourseDeltaLoader.failed_records_df, failed_records_df_new], ignore_index=True)
            
            # Log success message
            logging.info(f"Failed records stored successfully. Total failed records: {len(DoceboCourseDeltaLoader.failed_records_df)}")
            
            # Return updated DataFrame
            return DoceboCourseDeltaLoader.failed_records_df
        except Exception as e:
            # Log error if storing failed records fails
            logging.error(f"Failed to store failed records: {e}")
            return None
                
    def doccebo_course_delta_job(self):
        data = self.fetch_doccebo_course_data()
        df = self.preprocessing(data)
        result = self.load_doccebo_course_data(df, 'docebo-course-v2')
        return result


In [7]:
def fetch_existing_documents(client, index_name, document_ids):
    try:
        response = client.mget(index=index_name, body={"ids": document_ids})
        return {doc["_id"]: doc["found"] for doc in response["docs"]}
    except Exception as e:
        raise e

In [9]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_doccebo_course_data():
    # Define API endpoint URL
    base_url = "https://hp-gsb.docebosaas.com/learn/v1/courses"
    # Define parameters (filters)
    params = {
        #"updated_from": DoceboCourseDeltaLoader.fromDate,
        "page_size": 10,
        'adv': '1'
    }

    docebo_bearer = doccebo_get_access_token()

    # Make initial request to determine total pages
    initial_response = requests.get(base_url, params=params, headers={"Authorization": "Bearer " + docebo_bearer})
    initial_response.raise_for_status()
    initial_data = initial_response.json()
    total_pages = initial_data['data']['total_page_count']
    data = initial_data['data']['items']  # Collect the initial page

    def fetch_page_data(page):
        """Fetch a single page of Docebo course data."""
        page_params = params.copy()
        page_params['page'] = page
        response = requests.get(base_url, params=page_params, headers={"Authorization": "Bearer " + docebo_bearer})
        response.raise_for_status()
        return response.json()['data']['items']

    # Use ThreadPoolExecutor to fetch multiple pages in parallel
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_page_data, page) for page in range(2, total_pages + 1)]
        for future in as_completed(futures):
            data.extend(future.result())

    return data
import time
start=time.time()
check=fetch_doccebo_course_data()
end=time.time()
print(end-start)


257.48628091812134


In [8]:
result = DoceboCourseDeltaLoader()

In [9]:
import logging

# Set the logging level to WARNING to suppress DEBUG messages
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Example of your code here


In [10]:
data =result.fetch_doccebo_course_data()

In [11]:
len(data)

10296

In [12]:
df = result.preprocessing(data)

C:\Users\KuBP064\AppData\Local\Temp\ipykernel_5672\1605378819.py:73: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['description'] = df['description'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text(separator="\n").strip().replace("\n", " ").replace('•', '\n•'))


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3472 entries, 0 to 3471
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   documentID         3472 non-null   int64  
 1   title              3472 non-null   object 
 2   id                 3472 non-null   object 
 3   contentUpdateDate  3472 non-null   object 
 4   description        3472 non-null   object 
 5   contentType        3472 non-null   object 
 6   selling            3472 non-null   bool   
 7   code               3472 non-null   object 
 8   slug_name          3472 non-null   object 
 9   image              3472 non-null   object 
 10  language           3472 non-null   object 
 11  language_label     3472 non-null   object 
 12  price              3472 non-null   object 
 13  is_new             3472 non-null   object 
 14  is_opened          3472 non-null   object 
 15  current_rating     3472 non-null   float64
 16  credits            3472 

In [14]:
df1=result.get_persona(df)

{'data': {'access_token': '337c214a670fd1c61d221e52bb4a92a8e8e9a9d2', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}


In [15]:
df1.info()
# df1[df1['documentID']==12299].to_dict(orient='records')

<class 'pandas.core.frame.DataFrame'>
Index: 2975 entries, 0 to 3471
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   documentID         2975 non-null   int64  
 1   title              2975 non-null   object 
 2   id                 2975 non-null   object 
 3   contentUpdateDate  2975 non-null   object 
 4   description        2975 non-null   object 
 5   contentType        2975 non-null   object 
 6   selling            2975 non-null   bool   
 7   code               2975 non-null   object 
 8   slug_name          2975 non-null   object 
 9   image              2975 non-null   object 
 10  language           2975 non-null   object 
 11  language_label     2975 non-null   object 
 12  price              2975 non-null   object 
 13  is_new             2975 non-null   object 
 14  is_opened          2975 non-null   object 
 15  current_rating     2975 non-null   float64
 16  credits            2975 non-n

In [124]:
df=result.preprocessing(Incremental_data)

ERROR:root:An error occurred during preprocessing: "None of [Index(['current_rating', 'max_attempts'], dtype='object')] are in the [columns]"


In [85]:
import requests

# Initial list of root category IDs
root_categories = ['313', '2', '181', '157', '192', '190']

# Dictionary to store id_categories for each category ID (with names)
l2 = {}

# Dictionary to map each subcategory to its root
category_root_map = {}

# Keep track of categories to process, initializing with root categories
categories_to_check = root_categories.copy()

while categories_to_check:  # Continue until all categories are checked
    new_categories_to_check = []  # To hold new child categories
    
    for i in categories_to_check:
        # Get access token
        access_token = get_access_token()  # Assuming this function is defined
        
        # Define the API endpoint for each category
        api_endpoint_1 = f'https://hp-gsb.docebosaas.com/learn/v1/categories/{i}'
        
        # Set headers with the access token
        headers_2 = {"Authorization": "Bearer " + access_token}
        
        # Fetch category data
        response = requests.get(api_endpoint_1, headers=headers_2)
        data = response.json()
        
        # Extract 'idCategory' values (child categories) and their names from the response
        categories = data.get('data', [])
        if categories:  # If child categories are found
            if i not in l2:  # Add the parent category to l2 if not already present
                l2[i] = []
                
            # Iterate through the categories to capture both idCategory and category_name
            for category in categories:
                category_id = category['idCategory']
                category_name = category.get('category_name', 'Unknown')
                
                # Append category information (id and name) to the parent category
                l2[i].append({
                    'category_id': category_id,
                    'category_name': category_name
                })
                
                new_categories_to_check.append(category_id)  # Add children to the list for next iteration
                
                # Map all child categories to their respective root category
                root_category = category_root_map.get(i, i)  # Use the current category's root if already mapped
                category_root_map[category_id] = root_category  # Assign the root category to the child
        else:
            # If no child categories are found, mark it as finished and show the category ID
            print(f"Category {i} has no more child categories.")
            if i not in l2:  # If this category has no entry in l2, create an empty list
                l2[i] = []
    
    # Update categories to check with new child categories
    categories_to_check = new_categories_to_check

# l2 now contains a list of dictionaries for each parent category, with category_id and category_name


{'data': {'access_token': '3e9684c417bc3abb315fc65bbe2ce5caed66ee27', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}
{'data': {'access_token': 'cd1cae9ea2626927231ced286a5106526f6e99fd', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}
{'data': {'access_token': 'e378fb923ff2d2b5280ca9571e60967d374d6c7f', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}
{'data': {'access_token': '77e2b634b7eebe09e04f5aebe1ab1a363781fb67', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expi

In [197]:
# l2
# category_root_map
transformed_dict = {}

# Iterate over the original dictionary
for key, value in category_root_map.items():
    if value not in transformed_dict:
        transformed_dict[value] = []
    transformed_dict[value].append(key)
print(transformed_dict)

{'313': ['314', '315', '316', '317'], '2': ['34', '38', '39', '159', '165', '213', '294', '309', '62', '72', '118', '145', '148', '164', '188', '592', '49', '50', '53', '54', '63', '69', '152', '155', '156', '158', '160', '162', '186', '205', '285', '331', '392', '393', '394', '398', '539', '540', '596', '597', '598', '238', '358', '13', '41', '70', '120', '183', '282', '468', '541', '601', '189', '341', '295', '296', '297', '299', '300', '301', '303', '318', '577', '578', '579', '58', '151', '15', '146', '191', '149', '150', '244', '516', '140', '448', '508', '513', '542', '141', '411', '437', '511', '524', '413', '484', '525', '509', '166', '429', '430', '431', '432', '436', '433', '421', '470', '239', '5', '19', '242', '302', '304', '517', '449', '514', '438', '414', '415', '416', '417', '418', '419', '420', '434', '485', '422', '423', '425', '426', '427', '428', '435', '495', '471', '472', '473', '474', '475', '476', '477', '455', '543', '454', '462', '486', '450', '451', '452', '5

In [89]:
result.load_doccebo_course_data_test(df1,'docebo.v5.dev')

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urlli

(2974, 0)

In [472]:
prod_map=pd.read_excel('docebo_products.xlsx',sheet_name='Sheet2')
prod_map=prod_map[['seriesname','seriespressname']]
prod_map=prod_map.drop_duplicates(subset='seriesname').reset_index(drop=True)

In [479]:
prod_map1=pd.read_excel('docebo_products.xlsx',sheet_name='Sheet3')

In [147]:
access_token=get_access_token()
for i in df['documentID']:
    api_endpoint_1 = f'https://hp-gsb.docebosaas.com/learn/v1/courses/{i}'
    headers_2 = {"Authorization": "Bearer " + access_token}
    response_2 = requests.get(api_endpoint_1, headers=headers_2)
    response = response_2.json()
    # print(response)
    data = response['data']["additional_fields"]
    press_name_dict = next((item for item in data if item['title'] == 'Attendee Type'), None)
    print(i,press_name_dict)
    break

{'data': {'access_token': '0c35f7eda32c2cc3855c191b82f958d2c06c7538', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}
2 {'id': '1', 'title': 'Attendee Type', 'type': 'dropdown', 'visible_to_user': '0', 'settings': '[]', 'mandatory': True, 'value': 'HP Employee / Staff'}


In [161]:
### press_name from source
attendee={}
access_token=get_access_token()
for i in l:
    api_endpoint_1 = f'https://hp-gsb.docebosaas.com/learn/v1/courses/{i}'
    headers_2 = {"Authorization": "Bearer " + access_token}
    response_2 = requests.get(api_endpoint_1, headers=headers_2)
    #print("Status Code", response_2.status_code)
    #print("Response:", response_2.json())
    response = response_2.json()
    data = response['data']["additional_fields"]
    press_name_dict = next((item for item in data if item['title'] == 'Attendee Type'), None)
    attendee[i] = {
                'title': press_name_dict['title'],
                'value': press_name_dict['value']
            }
    #print('title',press_name_dict['title'],'value',press_name_dict['value'])
    #print(data)

{'data': {'access_token': 'f042cdcb86762de962a7d48a2d4ce1c749114c77', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}


KeyError: 'data'

In [168]:
attendee = {}
access_token = get_access_token()

for i in l:
    api_endpoint_1 = f'https://hp-gsb.docebosaas.com/learn/v1/courses/{i}'
    headers_2 = {"Authorization": "Bearer " + access_token}
    
    # Make the request
    response_2 = requests.get(api_endpoint_1, headers=headers_2)
    
    # Process the response if it's successful (status code 200)
    if response_2.status_code == 200:
        try:
            response = response_2.json()
            data = response['data'].get("additional_fields", [])
            press_name_dict = next((item for item in data if item['title'] == 'Attendee Type'), None)
            
            # Handle the case where 'Attendee Type' might not be present
            if press_name_dict:
                attendee[i] = {
                    'title': press_name_dict.get('title', 'N/A'),
                    'value': press_name_dict.get('value', 'N/A')
                }
            else:
                print(f"Course ID: {i} - 'Attendee Type' not found in additional fields")
                attendee[i] = {'title': 'Attendee Type', 'value': 'N/A'}
        except ValueError:
            print(f"Course ID: {i} - Failed to parse JSON response")
    else:
        print(f"Course ID: {i} - Request failed with status code {response_2.status_code}")
        attendee[i] = {'title': 'Attendee Type', 'value': 'N/A'}

# Optional: print the final attendee dictionary
print(attendee)

{'data': {'access_token': 'f09f05564d9f00ba53e4af72417e8e9f75d8de13', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}
Course ID: 2 - Status Code: 200
Course ID: 3 - Status Code: 200
Course ID: 7 - Status Code: 200
Course ID: 10 - Status Code: 200
Course ID: 12 - Status Code: 200
Course ID: 14 - Status Code: 200
Course ID: 15 - Status Code: 200
Course ID: 16 - Status Code: 200
Course ID: 17 - Status Code: 200
Course ID: 18 - Status Code: 200
Course ID: 20 - Status Code: 200
Course ID: 30 - Status Code: 200
Course ID: 53 - Status Code: 200
Course ID: 57 - Status Code: 200
Course ID: 69 - Status Code: 200
Course ID: 73 - Status Code: 200
Course ID: 74 - Status Code: 200
Course ID: 79 - Status Code: 200
Course ID: 80 - Status Code: 200
Course ID: 81 - Status Code: 200
Course ID: 82 - Status Code: 200
Course ID: 83 - Status Code: 200
Course ID: 84 -

In [110]:
import requests
import time 
import concurrent.futures

# Function to make the API request and process the response
def fetch_course_data(i, access_token):
    api_endpoint_1 = f'https://hp-gsb.docebosaas.com/learn/v1/courses/{i}'
    headers_2 = {"Authorization": "Bearer " + access_token}
    
    response_2 = requests.get(api_endpoint_1, headers=headers_2)   
    if response_2.status_code == 200:
        response = response_2.json()
        data = response['data'].get("additional_fields", [])
        is_published=response['data'].get('is_published','N/A')
        press_name_dict = next((item for item in data if item['title'] == 'Attendee Type'), None)
        
        if press_name_dict:
            return i, {'title': press_name_dict.get('title', 'N/A'),
                       'value': press_name_dict.get('value', 'N/A'),
                      'is_published':is_published}
        else:
            print(f"Course ID: {i} - 'Attendee Type' not found in additional fields")
            return i, {'title': 'Attendee Type', 'value': 'N/A'}
    else:
        print(f"Course ID: {i} - Request failed with status code {response_2.status_code}")
        return i, {'title': 'Attendee Type', 'value': 'N/A'}

# Main function to run API requests in parallel
def fetch_all_courses(df, access_token):
    l=[i for i in df['documentID']]
    attendee = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=25) as executor:
        # Map course IDs to fetch_course_data function
        futures = {executor.submit(fetch_course_data, i, access_token): i for i in l}
        
        for future in concurrent.futures.as_completed(futures):
            course_id, result = future.result()
            attendee[course_id] = result
    attendee_df=pd.DataFrame([(k, v['value'], v['is_published']) for k, v in attendee.items()], columns=['documentID', 'attendee_type', 'is_published'])       
    return attendee_df

# Example usage
access_token = get_access_token()
attendee = fetch_all_courses(df, access_token)


{'data': {'access_token': 'bcf085fc3f57973ba4069d7f6bebdf5d6463dff2', 'expires_in': '10800', 'token_type': 'bearer', 'scope': 'api', 'id_user': 208556, 'has_unfilled_fields': False, 'reenter_cc': False, 'trial_expired': False}, 'version': '1.0.0', '_links': []}


KeyboardInterrupt: 

In [14]:
l=[i for i in df['documentID']]

In [124]:
def fetch_all_courses(df):
    course_ids = df['documentID'].tolist()
    attendee = {}
    
    # Function to make the API request and process the response for each course
    def fetch_course_data(i):
        api_endpoint = f'https://hp-gsb.docebosaas.com/learn/v1/courses/{i}'
        headers = {"Authorization": "Bearer " + access_token}

        response = requests.get(api_endpoint, headers=headers)
        if response.status_code == 200:
            response_json = response.json()
            data = response_json['data'].get("additional_fields", [])
            is_published = response_json['data'].get('is_published', 'N/A')
            press_name_dict = next((item for item in data if item['title'] == 'Attendee Type'), None)

            if press_name_dict:
                return i, {
                    'title': press_name_dict.get('title', 'N/A'),
                    'value': press_name_dict.get('value', 'N/A'),
                    'is_published': is_published
                }
            else:
                print(f"Course ID: {i} - 'Attendee Type' not found in additional fields")
                return i, {'title': 'Attendee Type', 'value': 'N/A'}
        else:
            print(f"Course ID: {i} - Request failed with status code {response.status_code}")
            return i, {'title': 'Attendee Type', 'value': 'N/A'}

    # Parallel processing using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=25) as executor:
        futures = {executor.submit(fetch_course_data, course_id): course_id for course_id in course_ids}

        for future in concurrent.futures.as_completed(futures):
            course_id, result = future.result()
            attendee[course_id] = result

    # Creating a DataFrame with results
    attendee_df = pd.DataFrame(
        [(k, v['value'], v.get('is_published', 'N/A')) for k, v in attendee.items()],
        columns=['documentID', 'attendee_type', 'is_published']
    )
    updated_df=pd.merge(df,attendee_df1,how="left",left_on="documentID",right_on="documentID")
    updated_df['persona']=updated_df['attendee_type'].apply(lambda x:'Engineer' if x=='HP Employee / Staff' else 'Operator' if x=='Customer' else 'No_roles')
    updated_df['persona'] = updated_df.apply(lambda row: 
        "Engineer" if row['persona'] == 'No_roles' and (re.search(r'(\bce\b|\b-fse\b|\bce\b)', row['slug_name'].lower()) or 'service-engineer' in row['slug_name'].lower()) 
        else "Operator" if row['persona'] == 'No_roles' 
        else row['persona'], axis=1) 
    updated_df=updated_df[updated_df['is_published']==True]
    return updated_df
updated_df=fetch_all_courses(df)

In [35]:
# attendee
attendee_df1=pd.DataFrame([(k, v['value'], v['is_published']) for k, v in attendee.items()], columns=['documentID', 'attendee_type', 'is_published'])
attendee_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3467 entries, 0 to 3466
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   documentID     3467 non-null   int64 
 1   attendee_type  2276 non-null   object
 2   is_published   3467 non-null   bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 57.7+ KB


In [125]:
updated_df.info()
# updated_df['is_published']==

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3467 entries, 0 to 3466
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   documentID         3467 non-null   int64  
 1   title              3467 non-null   object 
 2   id                 3467 non-null   object 
 3   contentUpdateDate  3467 non-null   object 
 4   description        3467 non-null   object 
 5   contentType        3467 non-null   object 
 6   selling            3467 non-null   bool   
 7   code               3467 non-null   object 
 8   slug_name          3467 non-null   object 
 9   image              3467 non-null   object 
 10  language           3467 non-null   object 
 11  language_label     3467 non-null   object 
 12  price              3467 non-null   object 
 13  is_new             3467 non-null   object 
 14  is_opened          3467 non-null   object 
 15  current_rating     3467 non-null   float64
 16  credits            3467 

In [115]:
updated_df['persona']=updated_df['attendee_type'].apply(lambda x:'Engineer' if x=='HP Employee / Staff' else 'Operator' if x=='Customer' else None)

In [117]:
updated_df['persona']=updated_df['attendee_type'].apply(lambda x:'Engineer' if x=='HP Employee / Staff' else 'Operator' if x=='Customer' else None)
updated_df['persona'] = updated_df.apply(lambda row: 
    "Engineer" if row['persona'] == 'No_roles' and (re.search(r'(\bce\b|\b-fse\b|\bce\b)', row['slug_name'].lower()) or 'service-engineer' in row['slug_name'].lower()) 
    else "Operator" if row['persona'] == 'No_roles' 
    else row['persona'], axis=1)   

In [67]:
df[df['persona']=='Engineer'].loc[df.apply(lambda x:x['persona']!=x['persona1'],axis=1)][['documentID','title','persona','persona','slug_name','attendee_type']].to_dict(orient='records')

[{'documentID': 73,
  'title': 'HP Indigo Service Center FSE - AE hands-on exercises',
  'persona': 'Engineer',
  'persona_new': None,
  'slug_name': 'hp-indigo-service-center-fse-ae-hands-on-exercises',
  'attendee_type': None},
 {'documentID': 132,
  'title': 'HP Indigo 10000 Digital Press 2017 Delivery Program - CE',
  'persona': 'Engineer',
  'persona_new': None,
  'slug_name': 'hp-indigo-10000-digital-press-2017-delivery-program-ce',
  'attendee_type': None},
 {'documentID': 252,
  'title': "HP Indigo HP SmartStream L&P Print Server powered by Esko - What's New in v5.1 (CE and OP) Italian",
  'persona': 'Engineer',
  'persona_new': None,
  'slug_name': 'hp-indigo-hp-smartstream-lp-print-server-powered-by-esko-whats-new-in-v51-ce-and-op-italian',
  'attendee_type': None},
 {'documentID': 253,
  'title': "HP Indigo HP SmartStream L&P Print Server powered by Esko - What's New in v5.1 (CE and OP) German",
  'persona': 'Engineer',
  'persona_new': None,
  'slug_name': 'hp-indigo-hp-sma

In [129]:
updated_df.apply(lambda row: 
    "Engineer" if row['persona'] == 'Operator' and (re.search(r'(\bce\b|\b-fse\b|\bce\b)', row['slug_name'].lower()) or 'service-engineer' in row['slug_name'].lower()) 
    else "Operator", axis=1).value_counts()

Operator    3383
Engineer      84
Name: count, dtype: int64

In [1030]:
press_name_products=pd.read_csv('press_name_products.csv')

In [279]:
df0=pd.read_excel(r"C:\Users\KuBP064\Downloads\Courses and LP mappings data_11Mar2024.xlsx",sheet_name='GenericDeviceMappingOnCourse')
df11=pd.read_excel(r"C:\Users\KuBP064\Downloads\Courses and LP mappings data_11Mar2024.xlsx",sheet_name='GenericSeriesMappingOnCourse')
df12=pd.read_excel(r"C:\Users\KuBP064\Downloads\Courses and LP mappings data_11Mar2024.xlsx",sheet_name='DeviceCourseMapping')
df12 = df12.rename(columns={' CourseId': 'CourseId','ModelNameOrSkillName':'Products'})
df0 = df0.rename(columns={'# ModelNameOrSkillName':'Products'})
df11 = df11.rename(columns={'# SeriesName':'Products'})
df0 = df0.drop(columns='CourseName')
df11 = df11.drop(columns='CourseName')
df12 = df12.drop(columns='CourseName')
df13=pd.concat([df0,df11,df12])
df13['Products'] = df13.groupby('CourseId')['Products'].transform(lambda x: ', '.join(x))
df13.drop_duplicates('CourseId')
df13 = df13.drop_duplicates(subset='CourseId')
df13 = df13[['CourseId','Products']]
df13 = df13.sort_values(by='CourseId')
df13 = df13.rename(columns = {'CourseId':'documentID'})
df13 = df13.reset_index(drop=True)
df13.to_excel('doccebo_products.xlsx')
#products_mapping = pd.read_excel('doccebo_products.xlsx')
#df14 = pd.merge(df9,products_mapping, how='left', left_on='id_course', right_on='CourseId') 

In [26]:
def fetch_opensearch_data(client,documentID,index_name ):
    query = {
        "query": {
            "match": {
                "documentID": documentID
            }
        }
    }
    result = client.search(index=index_name, body=query)
    item_list = [hit for hit in result['hits']['hits']]
    return item_list

In [158]:
import pandas as pd

class DataQualityProcessor:

    @staticmethod
    def get_value(row, column):
        """ Helper method to safely get a value from a row, returning an empty string if the column is missing. """
        return row[column] if column in row else ''

    @staticmethod
    def generate_remarks(row, include_id=False, include_catalog_number=False):
        remarks = {
            'documentID': DataQualityProcessor.get_value(row, 'documentID'),
            'title': DataQualityProcessor.get_value(row, 'title'),
            'products': DataQualityProcessor.get_value(row, 'products'),
            'persona': DataQualityProcessor.get_value(row, 'persona'),
            'description': DataQualityProcessor.get_value(row, 'description'),
            'contentUpdateDate': DataQualityProcessor.get_value(row, 'contentUpdateDate'),
            'Domain': DataQualityProcessor.get_value(row, 'Domain'),
            'Doc_Status': DataQualityProcessor.get_value(row, 'Doc_Status'),
            'language': DataQualityProcessor.get_value(row, 'language'),
            'Title_is_too_short': 'N',
            'Description_is_too_short': 'N',
            'Domain_is_empty': 'N',
            'Title_is_empty': 'N',
            'Description_is_empty': 'N',
            'Disclosure_Level_is_empty': 'N',
            'Persona_is_empty': 'N',
            'Products_is_empty': 'N',
            'catalog_number_is_empty': 'N'
        }
        
        # Assign 'id' based on include_id flag
        remarks['id'] = DataQualityProcessor.get_value(row, 'id') if include_id else None
    
        # Assign 'catalog_number' based on include_catalog_number flag
        remarks['catalog_number'] = DataQualityProcessor.get_value(row, 'catalog_number') if include_catalog_number else None
    
        # Check if catalog_number is empty
        if 'catalog_number' in row and row['catalog_number'] == '':
            remarks['catalog_number_is_empty'] = 'Y'
    
        if len(remarks['title']) < 15 and len(remarks['title']) > 0:
            remarks['Title_is_too_short'] = 'Y'
    
        if len(remarks['description']) < 30 and len(remarks['description']) > 0:
            remarks['Description_is_too_short'] = 'Y'
    
        if remarks['Domain'] == '':
            remarks['Domain_is_empty'] = 'Y'
    
        if remarks['title'] == '':
            remarks['Title_is_empty'] = 'Y'
    
        if remarks['description'] == '':
            remarks['Description_is_empty'] = 'Y'
    
        if 'disclosureLevel' in row and row['disclosureLevel'] == '':
            remarks['Disclosure_Level_is_empty'] = 'Y'
    
        if 'persona' in row and row['persona'] == '':
            remarks['Persona_is_empty'] = 'Y'
    
        if 'products' in row and (row['products'] is None or len(row['products']) == 0):
            remarks['Products_is_empty'] = 'Y'
    
        return remarks
        
    @staticmethod   
    def process_dataframe(df,index_name,include_id, include_catalog_number):
        
        remarks_info = df.apply(DataQualityProcessor.generate_remarks, axis=1, include_id=include_id, include_catalog_number=include_catalog_number)
        df_remarks = pd.DataFrame(remarks_info.tolist())
        df_remarks["Index_Name"] =index_name
        df_remarks["Source"]= index_name.split('.')[0]
        df_remarks['contentUpdateDate'] = pd.to_datetime(df_remarks['contentUpdateDate']).dt.strftime('%Y-%m-%d %H:%M:%S')
        df_remarks.fillna('',inplace=True)
        df_remarks['Contents'] = df_remarks.apply(lambda row: {
            'title': row['title'],
            'products': row['products'],
            'persona': row['persona'],
            'description': row['description'],
            'contentUpdateDate': row['contentUpdateDate'],
            'Domain': row['Domain']
        }, axis=1)
        df_remarks['Contents']=df_remarks['Contents'].apply(lambda x:str(x))
        df_remarks=df_remarks.drop(columns=['products', 'persona'])
        columns_to_check = [
            "Title_is_too_short",
            "Description_is_too_short",
            "Domain_is_empty",
            "Title_is_empty",
            "Description_is_empty",
            "Disclosure_Level_is_empty",
            "Persona_is_empty",
            "Products_is_empty"
        ]
        if index_name.split('.')[0] =='kz':
            columns_to_check.append('catalog_number_is_empty')

        # Create the 'issues' column
        df_remarks['contains_issue'] = df_remarks[columns_to_check].apply(lambda row: 'Y' if 'Y' in row.values else 'N', axis=1)
        
        if 'kaas' in index_name:
            mask = df_remarks['documentID'].str.contains('ish') & (df_remarks['Title_is_too_short'] == 'Y')
            # Update the 'Title_is_too_short' column to 'N' for the filtered records
            df_remarks.loc[mask, 'Title_is_too_short'] = 'N'
            
        return df_remarks

    @staticmethod
    def generate_qc_data_log(test_df, index_name):
        index_name = index_name
        columns_to_check = [
            "Title_is_too_short",
            "Description_is_too_short",
            "Domain_is_empty",
            "Title_is_empty",
            "Description_is_empty",
            "Disclosure_Level_is_empty",
            "Persona_is_empty",
            "Products_is_empty",
            "contains_issue"
        ]
        
        # Append 'catalog_number_is_empty' if index_name starts with 'kz'
        if index_name.startswith('kz'):
            columns_to_check.append('catalog_number_is_empty')
        
        # Create dictionary to store counts using lambda
        count_dict = {
        col: test_df[col].value_counts().get('Y', 0) 
        for col in columns_to_check 
        if col in test_df.columns
        } 
        
        date_time = datetime.datetime.now()
        count_dict['datetime'] = date_time.strftime('%Y-%m-%d %H:%M:%S')
        count_dict['date'] = date_time.strftime('%Y-%m-%d')
        count_dict['source'] = index_name.split('.')[0]
        
        return count_dict

    @staticmethod
    def index_data_to_opensearch(df,index_name,qc_data_log):
        # Connect to OpenSearch
        config1 = OpenSearchConnectConfigStg()
        timeout_seconds = 160
        client = OpenSearch(
            hosts=[{'host': config1.host, 'port': config1.port}],
            http_compress=True,
            http_auth=config1.auth,
            use_ssl=config1.use_ssl,
            verify_certs=config1.verify_certs,
            timeout=timeout_seconds
        )
    
        # Retrieve data, preprocess, and convert to dictionary
    
        dict_list = df.to_dict(orient='records')
        print(dict_list)
        # Initialize counters and lists
        successful_records = 0
        failed_records = []
    
        # Prepare actions for bulk indexing
        actions = [
            {
                "_index": index_name,
                "_id": record["id"],
                "_op_type": "update",
                "doc": record,
                "doc_as_upsert": True  # Upsert if document doesn't exist
            }
            for record in dict_list
        ]
        
        # Perform bulk indexing
        try:
            success, _ = bulk(client, actions=actions)
            successful_records = success
        except Exception as e:
            # Log and store failed records
            for record in dict_list:
                failed_records.append((record, str(e), index_name.split('.')[0]))
                # logging.error(f"Failed to index document {record['_id']}: {e}")
        try:
            client.index(index='data_quality_check_datalog', body=qc_data_log)
        except Exception as e:
            raise
        return successful_records, len(failed_records)

In [159]:
test_df=DataQualityProcessor.process_dataframe(df1,'docebo.v5.dev',True,False)

In [160]:
test_df.to_dict(orient='records')[0]

{'documentID': 2,
 'title': 'HP Indigo FSE- AE Training for Service Center',
 'description': '<p>Learn how to use the new HP Indigo Service Center as a tool for managing services interactions and workflows, and resolving support cases, as relevant to your role.</p>\n<p>Duration: 120 min.</p>',
 'contentUpdateDate': '2022-04-19 20:26:58',
 'Domain': 'Indigo',
 'Doc_Status': 'published',
 'language': 'en',
 'Title_is_too_short': 'N',
 'Description_is_too_short': 'N',
 'Domain_is_empty': 'N',
 'Title_is_empty': 'N',
 'Description_is_empty': 'N',
 'Disclosure_Level_is_empty': 'N',
 'Persona_is_empty': 'N',
 'Products_is_empty': 'Y',
 'catalog_number_is_empty': 'N',
 'id': 'E-D19PVG',
 'catalog_number': '',
 'Index_Name': 'docebo.v5.dev',
 'Source': 'docebo',
 'Contents': "{'title': 'HP Indigo FSE- AE Training for Service Center', 'products': [], 'persona': 'Engineer', 'description': '<p>Learn how to use the new HP Indigo Service Center as a tool for managing services interactions and workf

In [161]:
#pd.set_option('display.max_colwidth', None)
import datetime
qc_data_log = DataQualityProcessor.generate_qc_data_log(test_df,'docebo.v5.dev')
qc_data_log

{'Title_is_too_short': 1,
 'Description_is_too_short': 25,
 'Domain_is_empty': 0,
 'Title_is_empty': 0,
 'Description_is_empty': 0,
 'Disclosure_Level_is_empty': 0,
 'Persona_is_empty': 0,
 'Products_is_empty': 1210,
 'contains_issue': 1221,
 'datetime': '2024-09-26 16:23:51',
 'date': '2024-09-26',
 'source': 'docebo'}

In [162]:
DataQualityProcessor.index_data_to_opensearch(test_df,'data_quality_check_iteration',qc_data_log)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.wa

(2972, 0)

In [164]:
from opensearchpy.helpers import scan
from opensearchpy import OpenSearch, helpers
class OpenSearchManager:
    config1=OpenSearchConnectConfigStg()
    client = OpenSearch(
                hosts=[{'host': config1.host, 'port': config1.port}],
                http_compress=True,
                http_auth=config1.auth,
                use_ssl=config1.use_ssl,
                verify_certs=config1.verify_certs
            )
    @staticmethod
    def fetch_and_update_missing_documents(index_name, document_id_series):
        documents_to_update = []
        query = {
            "query": {
                "match_all": {}
            }
        }

        # Fetch all documents from the index
        results = scan(OpenSearchManager.client, query=query, index=index_name)

        # Extract the document IDs from the DataFrame for comparison
        df_document_ids = document_id_series.astype(str).tolist()

        # Iterate over the fetched results
        for res in results:
            # If the documentID is not found in the DataFrame, mark it as 'rejected'
            if res['_id'] not in df_document_ids:
                # Remove the 'query_embedding' field if it exists
                res['_source'].pop('query_embedding', None)
                res['_source']['Doc_Status'] = 'deleted'
                documents_to_update.append(res)

        # Prepare the documents for bulk update
        actions = [
            {
                "_op_type": "update",
                "_index": index_name,
                "_id": doc['_id'],
                "doc": doc['_source']
            }
            for doc in documents_to_update
        ]

        # Perform the bulk update
        helpers.bulk(OpenSearchManager.client, actions)

        return f"{len(documents_to_update)} documents updated with 'deleted' in Doc_Status and 'query_embedding' removed in index '{index_name}'"

    @staticmethod
    def clean_dashboard_data(test_df,index_name,source):
        # index_name='data_quality_check_iteration'
        # source='kaas'
        query = {
            "query": {
                "match_all": {}
            }
        }
        results = scan(OpenSearchManager.client, query=query, index=index_name)
        test = [res['_source'] for res in results]
        cd_df = pd.DataFrame(test)
        cd_df = cd_df[cd_df['Source'] == source]
        
        # Identify unmatched records
        df_document_ids = test_df['documentID'].astype(str).tolist()
        cd_df['documentID'] = cd_df['documentID'].astype(str)
        
        # Records in cd_df but not in test_df
        unmatched_records = cd_df[~cd_df['documentID'].isin(df_document_ids)]
        actions = [
            {
                "_op_type": "update",
                "_index": index_name,
                "_id": row['documentID'],
                "doc": {
                    "Doc_Status": "deleted"
                }
            }
            for _, row in unmatched_records.iterrows()
        ]
        
        # Execute the bulk update
        if actions:
            helpers.bulk(OpenSearchManager.client, actions)
        return f"{len(actions)} documents updated with 'deleted' in Doc_Status and 'query_embedding' removed in index '{index_name}'"

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [165]:
import time
start_time = time.time()
print(OpenSearchManager.fetch_and_update_missing_documents('docebo.v5.dev', df1['id']))
end_time=time.time()
print('time_taken',end_time-start_time)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghr

3792 documents updated with 'deleted' in Doc_Status and 'query_embedding' removed in index 'docebo.v5.dev'
time_taken 35.727243185043335


In [166]:
import time
start_time = time.time()
print(OpenSearchManager.clean_dashboard_data(test_df,'data_quality_check_iteration', 'docebo'))
end_time=time.time()
print('time_taken',end_time-start_time)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghr

BulkIndexError: ('500 document(s) failed to index.', [{'update': {'_index': 'data_quality_check_iteration', '_id': '9092', 'status': 404, 'error': {'type': 'document_missing_exception', 'reason': '[9092]: document missing', 'index': 'data_quality_check_iteration', 'shard': '0', 'index_uuid': '7hS0J1-JRui2Dm44v7hZDw'}, 'data': {'doc': {'Doc_Status': 'deleted'}}}}, {'update': {'_index': 'data_quality_check_iteration', '_id': '9093', 'status': 404, 'error': {'type': 'document_missing_exception', 'reason': '[9093]: document missing', 'index': 'data_quality_check_iteration', 'shard': '0', 'index_uuid': '7hS0J1-JRui2Dm44v7hZDw'}, 'data': {'doc': {'Doc_Status': 'deleted'}}}}, {'update': {'_index': 'data_quality_check_iteration', '_id': '9094', 'status': 404, 'error': {'type': 'document_missing_exception', 'reason': '[9094]: document missing', 'index': 'data_quality_check_iteration', 'shard': '0', 'index_uuid': '7hS0J1-JRui2Dm44v7hZDw'}, 'data': {'doc': {'Doc_Status': 'deleted'}}}}, {'update':